### Import data and do a tiny bit of cleaning (on the 'column' column)

In [21]:
import pandas as pd
import numpy as np
from numpy import sin,cos,tan,cosh,sinh,tanh,deg2rad,sqrt

input_data = pd.read_csv("Service.csv", skiprows=6, skipfooter=8,
usecols=[0,2,4,5,6,7,8,9,10],
# na_values="-"
)
input_data.rename(columns={"Unnamed: 4": "load_combination","Unnamed: 2": "column","Unnamed: 0": "support", "Fvert\n[kN]":"Fv", "Fmajor\n[kN]":"Fy","Fminor\n[kN]":"Fx","Mmajor\n[kNm]":"My","Mminor\n[kNm]":"Mx","Mtor\n[kNm]":"Mtor"}, inplace=True)
input_data.ffill(inplace=True)

def column_sizes(input):
    """
    Function to re-write the column string in a more readable way
    """
    return input.split('(')[1].split(')')[0]
    
# Apply the column_sizes function to the imported data. 
input_data.column = input_data.column.apply(column_sizes)

# View the input_data
input_data.tail(20)


c:\Users\rmacdonald\OneDrive - Blyth & Blyth Consulting Engineers\Desktop\Work\Work 2\coding\.venv\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,support,column,load_combination,Fv,Fy,Fx,My,Mx,Mtor
1029,SUP K/#269,UKB 762x267x134,3 Dead,3.3,0.1,0.0,0.0,0.0,0.0
1030,SUP K/#269,UKB 762x267x134,4 Services,3.3,0.1,0.0,0.0,0.0,0.0
1031,SUP K/#269,UKB 762x267x134,5 Imposed,8.0,0.3,0.0,0.0,0.0,0.0
1032,SUP K/#269,UKB 762x267x134,6 Snow,9.5,0.3,0.0,0.0,0.0,0.0
1033,SUP K/#269,UKB 762x267x134,"8 Wind 0,Cpi -0.3,+Cpe",-4.9,0.0,5.8,0.0,0.0,0.0
1034,SUP K/#269,UKB 762x267x134,"9 Wind 0,Cpi -0.3,-Cpe",-4.9,0.0,5.8,0.0,0.0,0.0
1035,SUP K/#269,UKB 762x267x134,"10 Wind 0,Cpi 0.2,+Cpe",-13.4,-0.3,20.1,0.0,0.0,0.0
1036,SUP K/#269,UKB 762x267x134,"11 Wind 0,Cpi 0.2,-Cpe",-13.4,-0.3,20.1,0.0,0.0,0.0
1037,SUP K/#269,UKB 762x267x134,"12 Wind 90,Cpi -0.3,+Cpe",18.2,12.6,5.7,0.0,0.0,0.0
1038,SUP K/#269,UKB 762x267x134,"13 Wind 90,Cpi -0.3,-Cpe",9.7,8.8,5.7,0.0,0.0,0.0


### Support Class Definition

In [22]:

# define support class
class Support():
    def __init__(self, df, length, width):
        self.df = df
        self.calculate_initial_length_and_width()
        self.set_max_loading()        
        self.optimise_area(increment_size=0.01)

        self.starting_length = self.df
        self.bearing_pressure = None
        self.bearing_capacity = None 

    
    def calculate_initial_length_and_width(self,):
        """
        Uses the data in the df attribute to calculate initial length, width and area values and stores them within the object. 
        """
        assert len(self.df.column.unique()) == 1, print(f"Not all of the column values are the same")

        length = int(self.df.column.iloc[0][3:].split('x')[0])/1000
        width = int(self.df.column.iloc[0][3:].split('x')[1])/1000 #changing mm to m (/1000)
        self.length = length
        self.width = width
        self.area = length * width        

    def optimise_area(self, increment_size):
        self.bearing_capacity=150 # make input nicer... ask user for input?        
        self.bearing_pressure=self.calculate_bearing_pressure()
        print(f"Starting bearing pressure {self.bearing_pressure}, TRYING AREA {self.area}")
        iteration_number = 1 # track how many iterations we needed to get the optimal area...
        while self.bearing_pressure > self.bearing_capacity:
            iteration_number = iteration_number + 1
            self.length = self.length+increment_size
            self.width = self.width+increment_size
            self.area = self.width * self.length
            self.bearing_pressure = self.calculate_bearing_pressure()
        print(f"Found optimal area in {iteration_number} iterations")
        print(f"Calculated bearing pressure {self.bearing_pressure}, TRYING AREA {self.area:.2f} (L={self.length:.2f}m, W={self.width:.2f}m)")

        # can just do force/pressure to get maximum area then work out l, b beginning from there? most important bit is to figure out sliding
        # capacity for each of these, therefore will maybe be based on that
        # where should depth come into it - minimum depth that is okay for bearing pressure and sliding? Bearing pressure will never change, therefore can define that based on maximum FV, then use sliding to define L and B but always has to be less than calculated area, however depth can change
        print(f"Minimum length is {self.length:.2f}, minimum width is {self.width:.2f}\n") # 2f - format to 2 d.p.

    def loop_all_loading(self):
        self.set_loading()
        # bearing pressure calc to be renamed as 'set_important_parameters' or similar. 
        self.calculate_bearing_pressure()
        self.H_RES()
    def set_max_loading(self):
        """
        Based on the DF, set every load combination, then calculate the bearing pressure. Remember what the max loading condition is and set that. 
        """
        max_bearing_pressure = 0 # initialise as 0 
        for i in range(0,len(self.df)):
            self.P=self.df.iloc[i].Fv
            self.H_Y=self.df.iloc[i].Fy #df.FY
            self.H_X=self.df.iloc[i].Fx
            self.M_X=self.df.iloc[i].Mx
            self.M_Y=self.df.iloc[i].My
            # only update max_bearing pressure if it beats the last loading case... 
            bearing_pressure = self.calculate_bearing_pressure()
            if bearing_pressure > max_bearing_pressure:
                max_bearing_pressure = bearing_pressure
                max_loading_case = i 
                   
        print(f"max loading case {max_loading_case}")
        print(f"max bearing pressure {max_bearing_pressure}")
            
        self.P=self.df.iloc[max_loading_case].Fv
        self.H_Y=self.df.iloc[max_loading_case].Fy #df.FY
        self.H_X=self.df.iloc[max_loading_case].Fx
        self.M_X=self.df.iloc[max_loading_case].Mx
        self.M_Y=self.df.iloc[max_loading_case].My
        print(self.df.iloc[max_loading_case])
        # Now set the max loading on the object... 
        """
        USING calculated L and W, find self.H_RES and check it is greater than biggest horizontal load
        go through each loading case, find H_RES is H_RES < Fy is H_RES < FX then continue loop 
        when finished loop, print(f"Foundation PASSES for sliding")
        """
        for i in range(0,len(self.df)):
            self.P=self.df.iloc[i].Fv
            self.H_Y=self.df.iloc[i].Fy #df.FY
            self.H_X=self.df.iloc[i].Fx
            self.M_X=self.df.iloc[i].Mx
            self.M_Y=self.df.iloc[i].My
            # only update max_bearing pressure if it beats the last loading case... 
            resultant_sliding = self.H_RES
            if resultant_sliding < self.H_X:
                H=H+1
                print(f"FAIL")
            if resultant_sliding > self.H_Y:
                print(f"FAIL")

        



    def calculate_bearing_pressure(self):    
        # density of concrete (kN/m3)
        P_CONCRETE=23.6
        # depth of pad footing (m)
        H=0.750
        # depth of soil over pad (m)
        H_SOIL=0.500 #m
        P_SOIL=0.20 #m
        SHEAR_STRENGTH=deg2rad(25) #rad
        BASE_FRICTION=deg2rad(19.3) #rad
        P_BEARING=150 #kn/m2

        # column details
        I_A=0.850 #m base length
        B_A=0.450 #m base width
        E_PXA=0 #m eccentrincity in x
        E_PYA=0 #m ecc in y


        # Dead surcharge load (kN/m2)
        F_Gsur=0 
        F_Qsur=0

        # Pad footing self weight
        F_SWT=H*P_CONCRETE
        # soil self weight
        F_SOIL=H_SOIL*P_SOIL
        # total foundation load
        F=self.area*(F_SOIL+F_Qsur+F_SOIL+F_Gsur)



        # Stability against sliding (kN)
        H_friction=max((self.P+(F_Gsur+F_Qsur+F_SWT),0)) #resistance due to base friction
        K_P=(1+sin(SHEAR_STRENGTH))/(1-sin(SHEAR_STRENGTH)) #passive pressure coef
        H_D=sqrt(self.H_Y**2+self.H_X**2)

        # FIX H_X... 

        if self.H_X==0:
            ALPHA=0
        else:
            ALPHA=self.area*tan(self.H_Y/self.H_X)
        H_xpas=0.5*K_P*(H**2+2*H*H_SOIL)*self.width*P_SOIL  # BWCKY CHECK EQUATION IS STILL RIGHT!!
        H_ypas=0.5*K_P*(H**2+2*H*H_SOIL)*self.length*P_SOIL
        # resultant passive resistance
        H_RESULTANT=abs(H_xpas*cos(ALPHA))+abs(H_ypas*sin(ALPHA))
        
        # self.h_res_test()
        self.H_RES=H_friction+H_RESULTANT #should be greater than resultant horizontal load then PASS


        # STABILITY AGAINST TURNING
        # self.m_x_res_test() # check if fail or pass... 
        M_XOT=self.M_X+self.H_X*H
        M_XSUR=self.area*(F_Gsur+F_SWT+F_SOIL)*self.length/2
        M_XAXIAL=self.P*(self.length/2-E_PXA)
        M_XRES=M_XSUR+M_XAXIAL

        M_YOT=self.M_Y+self.H_Y*H
        M_YSUR=self.area*(F_Gsur+F_SWT+F_SOIL)*self.width/2
        M_YAXIAL=self.P*(self.width/2-E_PYA)
        M_YRES=M_YSUR+M_YAXIAL

        # Pad base reactions
        T=F+self.P
        E_TX=(self.P*E_PXA+self.M_X+self.H_X*H)/T
        E_TY=(self.P*E_PYA+self.M_Y+self.H_Y*H)/T

        Q_1=T/self.area-6*T*E_TX/(self.length*self.area)-6*T*E_TY/(self.width*self.area)
        Q_2=T/self.area-6*T*E_TX/(self.length*self.area)+6*T*E_TY/(self.width*self.area)
        Q_3=T/self.area+6*T*E_TX/(self.length*self.area)-6*T*E_TY/(self.width*self.area)
        Q_4=T/self.area+6*T*E_TX/(self.length*self.area)+6*T*E_TY/(self.width*self.area)

        Q_MIN=min(Q_1,Q_2,Q_3,Q_4)
        Q_MAX=max(Q_1,Q_2,Q_3,Q_4) #Is maximum base pressure less than the allowable
        bearing_pressure = Q_MAX
        return bearing_pressure


### Create the Support objects - and put them in the 'support_dictionary' for ease of access...

In [23]:
support_dictionary= {}
# New dataframes created for each different support type. 
# These dataframes are then used to create independent Support() objects, with different dataframes.

column_size=input_data.column.unique()

for support_name in column_size:
    independent_support_data=input_data.loc[input_data['column']==support_name]
    support_dictionary[support_name]=Support(df=independent_support_data, length=1000, width = 1000)

print(f"Support dict keys {support_dictionary.keys()}")


max loading case 194
max bearing pressure 6010.682041584373
support                                A10
column                     UKB 762x267x134
load_combination    8 Wind 0,Cpi -0.3,+Cpe
Fv                                   206.1
Fy                                    18.0
Fx                                   120.8
My                                     0.0
Mx                                     0.0
Mtor                                   0.0
Name: 194, dtype: object


UnboundLocalError: local variable 'H' referenced before assignment

##### Find max H-res, check if passes for sliiding in that direction, if doesnt increase depth until passes and THEN go back to bearing capacity with new pad depth and check that it still passes in bearing :) because when depth increases, max bearing pressure increases 

In [ ]:
# for key, support in support_dictionary.items():
#     # can loop by .keys(), .values(), or .items()...
#     # Our dict looks something like... {'UKB 762x267x134': Support(...)}
#     # key = 'UKB 762x267x134'
#     # values = Support(...)
#     # items returns both the key and value as a 'tuple'
#     print(key)
#     print(f"Area before optimisation = {support.area}")
#     support.optimise_area(increment_size = 0.01)
#     print(f"Area after optimisation = {support.area}\n")


# test_support = support_dictionary['UKB 457x191x67']
# len(test_support.df.column.unique())
# test_support.set_max_loading()   

# test_support.optimise_area(increment_size=0.01)

### Create a summary table

In [ ]:

# Empty df for storing the summary table
summary_table_df=pd.DataFrame()
# Get the largest Fv Fy and Fx and smallest Fv for all of the supports. 
for key, support in support_dictionary.items():
        for i in ['Fv','Fy','Fx']:
            if i == 'Fv':
                support.df.sort_values(by=i, ascending=False, inplace=True)
                summary_table_df = summary_table_df.append(support.df.iloc[0])
                support.df.sort_values(by=i, ascending=True, inplace=True)
                summary_table_df = summary_table_df.append(support.df.iloc[0])
            else:
                support.df.sort_values(by=i, ascending=False, inplace=True, key=abs)
                summary_table_df = summary_table_df.append(support.df.iloc[0])

summary_table_df.head(10)
# # assert len(summary_table_df) == 12, "Ahhhh noooo"
report_df=summary_table_df.copy()
report_df.set_index('column', inplace=True)
report_df.rename({'load_combination':'Load Combination'},axis='columns', inplace=True)
print(report_df.to_latex(columns=['Load Combination','Fv','Fy','Fx'],index_names=False))

\begin{tabular}{llrrr}
\toprule
{} &                 Load Combination &     Fv &    Fy &     Fx \\
\midrule
UKB 762x267x134 &           8 Wind 0,Cpi -0.3,+Cpe &  206.1 &  18.0 &  120.8 \\
UKB 762x267x134 &           10 Wind 0,Cpi 0.2,+Cpe & -326.6 & -29.3 & -118.1 \\
UKB 762x267x134 &          15 Wind 90,Cpi 0.2,-Cpe & -121.3 & -66.8 &    0.0 \\
UKB 762x267x134 &           9 Wind 0,Cpi -0.3,-Cpe &  206.1 &  18.0 &  120.8 \\
UKB 457x191x67  &         12 Wind 90,Cpi -0.3,+Cpe &   39.0 &  22.7 &   -0.1 \\
UKB 457x191x67  &          14 Wind 90,Cpi 0.2,+Cpe &  -43.8 &  33.5 &   -0.2 \\
UKB 457x191x67  &           9 Wind 0,Cpi -0.3,-Cpe &  -11.5 &  56.8 &    0.0 \\
UKB 457x191x67  &         12 Wind 90,Cpi -0.3,+Cpe &  -32.4 &  12.3 &   -0.2 \\
UC 203x203x60   &  1 Self weight - excluding slabs &    9.4 &   0.0 &    0.0 \\
UC 203x203x60   &          14 Wind 90,Cpi 0.2,+Cpe &   -9.9 &  -7.2 &  -13.4 \\
UC 203x203x60   &           11 Wind 0,Cpi 0.2,-Cpe &   -8.1 &  24.1 &   -5.8 \\
UC 203x203x6

c:\Users\rmacdonald\OneDrive - Blyth & Blyth Consulting Engineers\Desktop\Work\Work 2\coding\.venv\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
